In [1]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 13.7 MB/s eta 0:00:00


In [2]:
import os
import joblib
import faiss
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, AutoModel
import torch
import json


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
save_dir = "/content/drive/MyDrive/model_build/saved_model"
faiss_index_path = "/content/drive/MyDrive/model_build/faiss_index"
law_data_path = "/content/drive/MyDrive/Nvidia_Final_project/Data_labeled_demo/luatdat_labeled.json"


In [5]:
# Load FAISS Index
index = faiss.read_index(faiss_index_path)

In [6]:
# Load dữ liệu pháp luật JSON
with open(law_data_path, "r", encoding="utf-8") as f:
    law_data_json = json.load(f)
law_data = [item["text"] for item in law_data_json]

In [7]:
# Load tokenizer PhoBERT để tạo embeddings
phobert_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
phobert_model = AutoModel.from_pretrained("vinai/phobert-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [8]:
# Lớp Seq2Seq với Attention
class Seq2SeqAttention(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, hidden_size, **kwargs):
        super(Seq2SeqAttention, self).__init__(**kwargs)

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.encoder_lstm = tf.keras.layers.LSTM(hidden_size, return_sequences=True, return_state=True)
        self.decoder_lstm = tf.keras.layers.LSTM(hidden_size, return_sequences=True, return_state=True)
        self.attention = tf.keras.layers.Attention()
        self.dense = tf.keras.layers.Dense(vocab_size, activation="softmax")

        # Lưu tham số
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size

    def call(self, inputs):
        encoder_inputs, decoder_inputs = inputs

        encoder_embedded = self.embedding(encoder_inputs)
        encoder_outputs, state_h, state_c = self.encoder_lstm(encoder_embedded)

        decoder_embedded = self.embedding(decoder_inputs)
        decoder_outputs, _, _ = self.decoder_lstm(decoder_embedded, initial_state=[state_h, state_c])

        # Áp dụng Attention
        context_vector = self.attention([decoder_outputs, encoder_outputs, encoder_outputs])
        concat_outputs = tf.concat([decoder_outputs, context_vector], axis=-1)

        outputs = self.dense(concat_outputs)
        return outputs

    def get_config(self):
        config = super().get_config()
        config.update({
            "vocab_size": self.vocab_size,
            "embedding_dim": self.embedding_dim,
            "hidden_size": self.hidden_size
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)



model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [9]:
def encode_text(text):
    """Tạo embedding từ văn bản bằng PhoBERT"""
    inputs = phobert_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=256)
    with torch.no_grad():
        outputs = phobert_model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()  # Mean pooling
    return embedding


def search_law_text(query, top_k=3):
    """Tìm kiếm văn bản pháp luật liên quan bằng FAISS"""
    query_embedding = encode_text(query).reshape(1, -1)
    distances, indices = index.search(query_embedding, top_k)

    retrieved_texts = [law_data[i] for i in indices[0] if i < len(law_data)]
    return retrieved_texts



#def encode_text(text):
#    """Tạo embedding từ văn bản bằng PhoBERT"""
#   inputs = phobert_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=256)
#    with torch.no_grad():
#        outputs = phobert_model(**inputs)
 #   embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()  # Mean pooling
  #  return embedding


#def search_law_text(query, top_k=7):
 #   """Tìm kiếm văn bản pháp luật liên quan bằng FAISS"""
  #  query_embedding = encode_text(query).reshape(1, -1)
   # distances, indices = index.search(query_embedding, top_k)

    #retrieved_texts = [law_data[i] for i in indices[0] if i < len(law_data)]

    # Kết hợp các văn bản luật lại, nhưng giới hạn độ dài tránh input quá dài
    #combined_text = " ".join(retrieved_texts)
    #return combined_text[:4080]  # Cắt nếu quá dài


In [10]:
def generate_response(prompt, tokenizer, model, max_length=150, top_k=3):
    # 🔹 Truy xuất nhiều văn bản pháp luật
    retrieved_texts = search_law_text(prompt, top_k)

    # 🔹 Kết hợp tất cả các điều luật truy xuất
    retrieved_context = "\n".join([f"- {text}" for text in retrieved_texts])

    # 🔹 Đưa văn bản pháp luật vào đầu vào của mô hình Seq2Seq
    full_input = f"{prompt} {retrieved_context}"

    # Token hóa input
    encoded_prompt = tokenizer(full_input, return_tensors="tf", padding=True, truncation=True, max_length=max_length)
    input_ids = encoded_prompt["input_ids"]

    # Kiểm tra tokenizer có bos/eos token không
    start_token = tokenizer.bos_token_id or tokenizer.cls_token_id
    end_token = tokenizer.eos_token_id or tokenizer.sep_token_id

    # Khởi tạo decoder
    decoder_input = tf.convert_to_tensor([[start_token]], dtype=tf.int32)
    response_ids = []

    for _ in range(max_length):
        # Dự đoán từ tiếp theo
        predictions = model([input_ids, decoder_input], training=False)
        predicted_id = tf.argmax(predictions[:, -1, :], axis=-1).numpy()[0]

        if predicted_id == end_token:
            break

        response_ids.append(predicted_id)
        decoder_input = tf.concat([decoder_input, tf.convert_to_tensor([[predicted_id]], dtype=tf.int32)], axis=1)

    # 🔹 Sinh câu trả lời từ mô hình
    generated_response = tokenizer.decode(response_ids, skip_special_tokens=True)

    # 🔹 Trả về câu trả lời kèm trích dẫn điều luật
    return f"**Tóm tắt:** {generated_response}\n\n**Trích dẫn pháp luật:**\n{retrieved_context}"


'''
def generate_response(prompt, tokenizer, model, max_length=200):
    """Sinh câu trả lời từ chatbot bằng cách kết hợp FAISS + Seq2Seq"""
    # Truy xuất văn bản pháp luật từ FAISS
    retrieved_text = search_law_text(prompt)

    # Tạo đầu vào mới cho Seq2Seq (gồm cả prompt + kết quả tìm kiếm)
    full_input = f"{prompt} {retrieved_text}"

    # Token hóa input
    encoded_prompt = tokenizer(full_input, return_tensors="tf", padding=True, truncation=True, max_length=256)
    input_ids = encoded_prompt["input_ids"]

    # Kiểm tra tokenizer có bos/eos token không
    start_token = tokenizer.bos_token_id or tokenizer.cls_token_id
    end_token = tokenizer.eos_token_id or tokenizer.sep_token_id

    # Khởi tạo decoder
    decoder_input = tf.convert_to_tensor([[start_token]], dtype=tf.int32)
    response_ids = []

    for _ in range(max_length):
        # Dự đoán từ tiếp theo
        predictions = model([input_ids, decoder_input], training=False)
        predicted_id = tf.argmax(predictions[:, -1, :], axis=-1).numpy()[0]

        if predicted_id == end_token:
            break

        response_ids.append(predicted_id)
        decoder_input = tf.concat([decoder_input, tf.convert_to_tensor([[predicted_id]], dtype=tf.int32)], axis=1)

    # Giải mã output
    response_text = tokenizer.decode(response_ids, skip_special_tokens=True)
    return response_text
'''

'\ndef generate_response(prompt, tokenizer, model, max_length=200):\n    """Sinh câu trả lời từ chatbot bằng cách kết hợp FAISS + Seq2Seq"""\n    # Truy xuất văn bản pháp luật từ FAISS\n    retrieved_text = search_law_text(prompt)\n\n    # Tạo đầu vào mới cho Seq2Seq (gồm cả prompt + kết quả tìm kiếm)\n    full_input = f"{prompt} {retrieved_text}"\n\n    # Token hóa input\n    encoded_prompt = tokenizer(full_input, return_tensors="tf", padding=True, truncation=True, max_length=256)\n    input_ids = encoded_prompt["input_ids"]\n\n    # Kiểm tra tokenizer có bos/eos token không\n    start_token = tokenizer.bos_token_id or tokenizer.cls_token_id\n    end_token = tokenizer.eos_token_id or tokenizer.sep_token_id\n\n    # Khởi tạo decoder\n    decoder_input = tf.convert_to_tensor([[start_token]], dtype=tf.int32)\n    response_ids = []\n\n    for _ in range(max_length):\n        # Dự đoán từ tiếp theo\n        predictions = model([input_ids, decoder_input], training=False)\n        predicted_

In [11]:
# Load tokenizer cho Seq2Seq
tokenizer_path = os.path.join(save_dir, "tokenizer.pkl")
tokenizer = joblib.load(tokenizer_path)

In [12]:
# Load mô hình Seq2Seq
model = tf.keras.models.load_model(
    os.path.join(save_dir, "seq2seq_model.keras"),
    custom_objects={"Seq2SeqAttention": Seq2SeqAttention}
)

In [13]:
prompt = "Có được dùng đất nông nghiệp để xây nhà không?"


In [14]:
response = generate_response(prompt, tokenizer, model)

print("Bot trả lời:", response)


Bot trả lời: **Tóm tắt:** Không, trừ trường hợp được công trình thuộc diện miễn theo quy định của pháp luật.

**Trích dẫn pháp luật:**
- a) hộ gia đình, cá nhân đang sử dụng đất không phải là đất thuê trả tiền thuê đất hằng năm;
- b) trường hợp không còn hoặc thiếu đất nông nghiệp thì được giao tiếp đất nông nghiệp trong hạn mức không thu tiền hoặc cho thuê đất phi nông nghiệp không phải là đất ở để sản xuất, kinh doanh và được miễn, giảm tiền thuê đất.
- d) cho thuê đất phi nông nghiệp không phải là đất ở để sản xuất, kinh doanh và được miễn, giảm tiền thuê đất;
